In [ ]:
import numpy as np
import os
import pandas as pd
import progressbar
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.models import Sequential
from keras.utils import np_utils
from keras import backend as K
from urllib.request import urlopen
K.set_image_dim_ordering('th')

# For reproducibility, set the initial seed
seed = 7
np.random.seed(7)

def load_dataset(dataset_fp, delimiter=",",chunksize=1000):
    if not os.path.isfile(dataset_fp):
        response = urlopen("http://cs.mcgill.ca/~ksinha4/datasets/kaggle/" + dataset_fp)
        CHUNK = 16 * chunksize
        with open(dataset_fp, 'wb') as f:
            while True:
                chunk = response.read(CHUNK)
                if not chunk:
                    break
                f.write(chunk)
    
    
    chunks = []
    pb = progressbar.ProgressBar()
    for chunk in pb(pd.read_csv(dataset_fp, delimiter=delimiter, chunksize=chunksize, header=None)):
        chunks.append(chunk)
        
    dataset = pd.concat(chunks)
    return dataset.as_matrix()


def train_validation_set_split(trainset_x, trainset_y, **kwargs):
    trainset = np.concatenate((trainset_x, trainset_y),axis=1)
    return train_test_split(trainset, **kwargs)

In [ ]:
import skimage 
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from PIL import Image, ImageOps

def get_regions(image):
    bw = closing(image > 0.99, square(1))
    
    # label image regions
    label_image = label(bw)
    return [region.image for region in regionprops(label_image)]

def max_region_by_area(regions):
    return max(regions, key = lambda x : max(x.shape[0] * x.shape[0], x.shape[1] * x.shape[1]))


def to_squre(region):
    (w, h) = region.shape
    desired_size = 32
    delta_w = desired_size - h
    delta_h = desired_size - w
    padding = (delta_w//2, delta_h//2, delta_w-(delta_w//2), delta_h-(delta_h//2))
    im = Image.fromarray(region.astype('uint8')*255)
    new_im = ImageOps.expand(im, padding)
    im_array = np.array(new_im)
    transformed_im = skimage.transform.resize(im_array, (desired_size,desired_size))
    return transformed_im
    
    
def preprocess_image(image):
    p_image = image.reshape(64,64)
    p_image = p_image.astype('float32')
    regions = get_regions(p_image)
    max_area_region = max_region_by_area(regions)
    return to_squre(max_area_region)

In [ ]:
xtrainload = load_dataset("train_x.csv")
ytrainload = load_dataset("train_y.csv")

In [ ]:
xtrain = xtrainload / 255.0
ytrain = ytrainload 

In [ ]:
pb = progressbar.ProgressBar()


# preprocess x 
xtrain_preprocessed = []
for x in pb(xtrain):
    result = preprocess_image(x)
    result = result.reshape(1024)
    xtrain_preprocessed.append(result)
    
# Preprocess y 
ytrain = np_utils.to_categorical(ytrain)
xtrain = np.asarray(xtrain_preprocessed)
num_classes = ytrain.shape[1]


xtrainset = xtrain[:-10000]
ytrainset = ytrain[:-10000]
xvalidset = xtrain[-10000:]
yvalidset = ytrain[-10000:]

In [ ]:
def base_cnn():
    '''
    ConvNet1
    Following online tutorial found here:
    https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/
    '''
    # create model
    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=(1, 32, 32), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def lecun_cnn():
    '''
    ConvNet2
    Parameter choices inspired by the LeCun-5 paper which was optimized for performance on
    MNIST dataset.
    http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf
    '''
    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=(1, 32, 32), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def lecun_cnn_double_hidden_layers():
    '''
    ConvNet3
    Additional hidden layer was added 
    '''
    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=(1, 32, 32), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def lecun_cnn_double_hidden_layer_width():
    '''
    ConvNet4
    Parameter choices inspired by the LeCun-5 paper which was optimized for performance on
    MNIST dataset.
    http://yann.lecun.com/exdb/publis/pdf/lecun-98.pdf
    '''
    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=(1, 32, 32), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(360, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(360, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def lecun_cnn_double_convolution():
    '''
    ConvNet5
    '''
    # create model
    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=(1, 32, 32), activation='relu'))
    model.add(Conv2D(6, (5, 5), input_shape=(1, 32, 32), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), activation='relu'))
    model.add(Conv2D(16, (5, 5), input_shape=(1, 32, 32), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(180, activation='relu'))
    model.add(Dense(180, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def lecun_cnn_double_convolution_wider():
    '''
    ConvNet6
    '''
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(1, 32, 32), activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    



In [ ]:
trainxall = np.append(xtrainset, xvalidset)
trainyall = np.append(ytrainset, yvalidset)
trainxallshaped = trainxall.reshape(xtrainload.shape[0],1,32,32)
trainyallshaped = trainyall.reshape(ytrainload.shape[0],10)

In [ ]:
'''
Base CNN from tutorial
'''
base_model = base_cnn()
base_model.fit(xtrainset.reshape(xtrainset.shape[0],1,32,32),
          ytrainset, 
          validation_data=(xvalidset.reshape(xvalidset.shape[0],1,32,32), yvalidset),
          batch_size=200, 
          epochs=20, 
          verbose=2)

In [ ]:
'''
LeCun-5 Inspired cnn weights.
'''

lecun_model = lecun_cnn()
lecun_model.fit(xtrainset.reshape(xtrainset.shape[0],1,32,32),
          ytrainset, 
          validation_data=(xvalidset.reshape(xvalidset.shape[0],1,32,32), yvalidset),
          batch_size=200, 
          epochs=20, 
          verbose=2)


In [ ]:
'''
Doubling Number of Hidden Layers
'''
lecun_model_2 = lecun_cnn_double_hidden_layers()
lecun_model_2.fit(xtrainset.reshape(xtrainset.shape[0],1,32,32),
          ytrainset, 
          validation_data=(xvalidset.reshape(xvalidset.shape[0],1,32,32), yvalidset),
          batch_size=200, 
          epochs=20, 
          verbose=2)

In [ ]:
'''
Doubling Width of Hidden Layers
'''
lecun_model_3 = lecun_cnn_double_hidden_layers()
lecun_model_3.fit(xtrainset.reshape(xtrainset.shape[0],1,32,32),
          ytrainset, 
          validation_data=(xvalidset.reshape(xvalidset.shape[0],1,32,32), yvalidset),
          batch_size=200, 
          epochs=20, 
          verbose=2)

In [ ]:
'''
Doubling Convolution Layers and Width
'''
lecun_model_4 = lecun_cnn_double_convolution()
lecun_model_4.fit(xtrainset.reshape(xtrainset.shape[0],1,32,32),
          ytrainset, 
          validation_data=(xvalidset.reshape(xvalidset.shape[0],1,32,32), yvalidset),
          batch_size=200, 
          epochs=20, 
          verbose=2)

In [ ]:
'''
Increase width and number of convolutional layers, increase width of dense layers.
'''

lecun_model_5 = lecun_cnn_double_convolution_wider()
lecun_model_5.fit(xtrainset.reshape(xtrainset.shape[0],1,32,32),
          ytrainset, 
          validation_data=(xvalidset.reshape(xvalidset.shape[0],1,32,32), yvalidset),
          batch_size=200, 
          epochs=20, 
          verbose=2)

In [ ]:
xtestset = load_dataset("test_x.csv")
xtestset = xtestset / 255.

pb = progressbar.ProgressBar()

xtest_preprocessed = []
for x in pb(xtestset):
    result = preprocess_image(x)
    result = result.reshape(1024)
    xtest_preprocessed.append(result)

xtestset = np.asarray(xtest_preprocessed)
xtestset = xtestset.reshape(xtestset.shape[0],1,32,32)


In [ ]:
# Final Model, save results to upload to Kaggle. 

lecun_model = lecun_cnn_double_convolution_wider()
lecun_model.fit(trainxallshaped,
                trainyallshaped, 
                batch_size=200, 
                epochs=25, 
                verbose=2)


predictions = lecun_model.predict(xtestset)

with open('results.csv','w') as prediction_file:
    prediction_file.write("Id,Label\n")
    for i,prediction in enumerate(predictions):
        prediction_file.write("%d,%d\n"%(i,list(prediction).index(max(prediction))))
        

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils.vis_utils import plot_model

# Create diagrams of each CNN.

cnns = [base_cnn, 
        lecun_cnn, 
        lecun_cnn_double_convolution, 
        lecun_cnn_double_convolution_wider, 
        lecun_cnn_double_hidden_layer_width,
        lecun_cnn_double_hidden_layers]

for cnn in cnns:
    file_name = cnn.__name__ + ".png"
    plot_model(cnn(), to_file=file_name)
    

